# Imports

In [11]:
import requests
from flask import jsonify
import json
from datetime import date, time, datetime

# Testing POST /merge_pdfs endpoint

In [12]:
body_all = {"1": {"bp_bank_id": "0170",
             "last_updated": "2021-04-06",
             "url": "https://www.abanca.pt"
             },
             "2": {"bp_bank_id": "0269",
             "last_updated": "2021-04-06",
             "url": "https://www.bankinter.pt"
             },
             "3": {"bp_bank_id": "0079",
             "last_updated": "2021-04-06",
             "url": "https://www.bancobic.ao"
             },
             "4": {"bp_bank_id": "0008",
             "last_updated": "2021-04-06",
             "url": "https://www.bancobai.ao"
             },
             "5": {"bp_bank_id": "0193",
             "last_updated": "2021-04-06",
             "url": "https://www.bancoctt.pt/"
             }
            }

In [13]:
# requires this request body
body = {"1": {"bp_bank_id": "0170",
  "last_updated": "2021-04-06",
  "url": "https://www.abanca.pt"
  }
}

In [14]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/merge_pdfs"
url_h = 'https://bank-price-api.herokuapp.com/merge_pdfs'

In [15]:
response = requests.post(url, data = None, json = body).json()

In [16]:
ident = response.get('ident')

In [17]:
ident

488

## Testing GET /retrievepdfs

Since the ```/merge_pdfs``` job can take some time and we want to avoid heroku timeout issues the job is done in the background. To pick up the requested file, the rails app can go to this endpoint after some waiting time. 
There will be only one required argument ```ident``` to be passed to the GET request which was given before by the response of ```/merge_pdfs```.

In [18]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/retrievepdfs"
url_h = 'https://bank-price-api.herokuapp.com/retrievepdfs'

In [19]:
params = {'ident':ident}

In [20]:
# call this after logs say: 'updated bank.json ready for pickup with ident: <ident>'
response = requests.get(url, params=params).json()

In [21]:
response

{'1': {'bp_bank_id': '0170',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
  'last_updated': '2021-04-14',
  'list_pdfs': {'cloud_merged_url': 'https://storage.googleapis.com/bank_price_pdfs/1_all_products_210414110502.pdf',
   'cloud_url_size': '2548956',
   'urls': ['https://www.abanca.pt/files/documents/precario-folheto-comissoes-60ba13f9.pdf',
    'https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf',
    'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf',
    'https://www.abanca.pt/files/docs/precario-contasempresa-aviso8-2009.pdf',
    'https://www.abanca.pt/files/documents/glossario-6479e89f.pdf',
    'https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf']},
  'num_pdfs': 6,
  'price_page': 'https://www.abanca.pt/pt/precario/',
  'status': 'ok',
  'url': 'https://www.abanca.pt'}}

In [23]:
# saving infos for the other endpoints
bp_pdf_url = response.get('1').get('bp_pdf_url')
bp_bank_id = response.get('1').get('bp_bank_id')

# Testing POST /get_stats

In [24]:
# requires this request body
body = {
    "1": {
        "url": "www.abanca.pt",
        "bp_pdf_url": bp_pdf_url,
        "bp_bank_id": bp_bank_id,
        "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf",
        "products": {"demand_deposit":
                {
                    "statement": [
                    "Emissão de extrato",
                    "Extrato Integrado",
                    "Extrato Mensal",
                    "Extrato integrado",
                    "Extrato avulso"
                ],
                "documents_copy": [
                    "Fotocópias de segundas vias de talões de depósito",
                    "Emissão 2ªs Vias de Avisos e Outros Documentos",
                    "Extracto avulso",
                    "Segundas vias (pedido na agência)"
                ],
                "acc_manteinance": [
                    "Manutenção de conta",
                    "Comissão de manutenção de conta",
                    "Comissão de Manutenção de Conta",
                    "Manutenção de Conta Pacote",
                    "Manutenção de Conta Base",
                    "Manutenção de Conta Serviços Mínimos Bancários"
                ],
                "withdraw": [
                    "Levantamento de numerário",
                    "Levantamento de numerário ao balcão",
                    "Comissão de Levantamento",
                    "Levantamento de Numerário ao Balcão",
                    "Levantamento de Numerário ao Balcão"
                ],
                "online_service": [
                    "Adesão ao serviço de banca à distância",
                    "Adesão ao serviço online"
                ],
                "cash_deposit": [
                    "Depósito de moedas metálicas",
                    "Depósito de moedas",
                    " Depósito em moeda metálica (>= 100 moedas)",
                    "Depósito de moedas ao balcão",
                    "Depósito de dinheiro ao balcão",
                    "Depósito em moeda metálica (>= 100 moedas)"
                ],
                "change_holder": [
                    "Alteração de titulares",
                    "Alteração de titularidade",
                    "Comissão de Alteração de Titularidade",
                    "Alteração de titularidade / intervenientes",
                    "Alteração de titularidade (titular/ representante)"
                ],
                "bank_overdraft": [
                    "Comissões por descoberto bancário",
                    "Descoberto bancário",
                    "Comissões por Descoberto Bancário"
                ],
                "movement_consultation": [
                    "Consulta de Movimentos de conta DO com",
                    "Consulta de movimentos ao balcão"
                ],
                "balance_inquiry": [
                    "Pedido de saldo ao balcão",
                    "Consulta de Saldo de conta DO com comprovativo"
                ],
                "portuguese": "Contas de Depósito"
            }
        }
    }
}

In [25]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/get_stats"
url_h = 'https://bank-price-api.herokuapp.com/get_stats'

In [26]:
response = requests.post(url, data = None, json = body).json()

In [29]:
ident_s = response.get('ident')

In [30]:
ident_s

8334

## Testing GET /retrievestats

Since the ```/get_stats``` job can take some time and we want to avoid heroku timeout issues the job is done in the background. To pick up the requested file, the rails app can go to this endpoint after some waiting time. 
There will be only one required argument ```ident``` to be passed to the GET request which was given before by the response of ```/get_stats```.

In [9]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/retrievestats"
url_h = 'https://bank-price-api.herokuapp.com/retrievestats'

In [31]:
params = {'ident':ident_s}

In [ ]:
# call this after logs say: 'bank price json with <ident> loaded, supplied and removed from server'
response = requests.get(url, params=params).json()

In [ ]:
response